In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.pipeline import Pipeline
from feature_engine import outlier_removers as outr
from feature_engine.outlier_removers import Winsorizer
from feature_engine.categorical_encoders import OneHotCategoricalEncoder, RareLabelCategoricalEncoder

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline


## Import Data

In [2]:
# read in loan data
df = pd.read_csv('Data/loan.csv')
df.shape

(2260668, 145)

In [3]:
# read in column names and descriptions
dictionary = pd.read_csv('Data/LCDataDictionary.csv', names=['type','feature','description'])

# remove error in feature name
dictionary['feature'].replace('total_rev_hi_lim \xa0','total_rev_hi_lim',inplace=True)


# create list of feature names to be used in df
cols_keep= list(dictionary['feature'].values)

In [4]:
df = df[cols_keep]
# drop columns which have less than 65% available data
df.dropna(axis=1, how='any', thresh=int(0.65*len(df)), inplace=True)
df.shape

(2260668, 63)

Original df:  (2260668, 145)
Dropped <65% data:  (2260668,  63)


## Manipulating Labels

For our model we're predicting default rate. Only examining completed loans - loan_status equals charged off or fully paid


In [5]:
df.loan_status.unique()

array(['Current', 'Fully Paid', 'Late (31-120 days)', 'In Grace Period',
       'Charged Off', 'Late (16-30 days)', 'Default',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off'],
      dtype=object)

In [6]:
labels_to_keep = ['Fully Paid', 'Charged Off', 'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off']
df = df[df.loan_status.isin(labels_to_keep)]

status_dict = {'Does not meet the credit policy. Status:Fully Paid':'Fully Paid',
        'Does not meet the credit policy. Status:Charged Off':'Charged Off'}

df['loan_status'].replace(status_dict, inplace=True)
df['loan_status'].value_counts(normalize=True)

Fully Paid     0.799124
Charged Off    0.200876
Name: loan_status, dtype: float64

In [7]:
# delete all the columns that contain a single unique value
for col in df.columns:
    if len(df[col].unique()) == 1:
        print("Dropped " + col)
        df.drop(col, inplace=True, axis=1)
print(df.shape)

Dropped policy_code
Dropped pymnt_plan
(1306356, 61)


## Split data into train and test 

In [8]:
X = df.drop(columns = 'loan_status')
y = df.loan_status.values.ravel()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=13, stratify=None)

X_train.head()

,acc_now_delinq,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,delinq_amnt,dti,funded_amnt,funded_amnt_inv,...,home_ownership,initial_list_status,purpose,sub_grade,term,title,verification_status,hardship_flag,disbursement_method,debt_settlement_flag
2086726,0.0,60000.0,1122.0,20411.0,17.0,0.0,0.0,13.70,16000,16000.0,...,MORTGAGE,w,home_improvement,F2,60 months,Home improvement,Source Verified,N,Cash,N
787320,0.0,98000.0,3660.0,7743.0,35.5,1.0,0.0,6.93,10000,10000.0,...,RENT,w,debt_consolidation,B2,60 months,NaN,Not Verified,N,Cash,N
1680980,0.0,55000.0,24630.0,19.0,99.4,1.0,0.0,16.78,14550,14550.0,...,MORTGAGE,f,debt_consolidation,D4,60 months,Debit Loan,Source Verified,N,Cash,N
1240845,0.0,43500.0,25888.0,170.0,96.5,0.0,0.0,16.00,12000,12000.0,...,MORTGAGE,w,debt_consolidation,D4,60 months,Debt consolidation,Not Verified,N,Cash,N
1883061,0.0,40000.0,1320.0,1221.0,71.6,0.0,0.0,12.34,5600,5600.0,...,RENT,f,credit_card,B5,36 months,Credit card refinancing,Not Verified,N,Cash,N


## Dividing data into categorical and numerical parts¶


In [10]:
df.dtypes.unique()

array([dtype('float64'), dtype('int64'), dtype('O')], dtype=object)

In [11]:
numerical_cols_df = X_train.select_dtypes(include=['float64','int64'])
categorical_cols_df = X_train.select_dtypes(include='O')

num_cols = numerical_cols_df.columns
cat_cols = categorical_cols_df.columns

In [12]:
cat_cols 

Index(['addr_state', 'emp_length', 'emp_title', 'grade', 'home_ownership',
       'initial_list_status', 'purpose', 'sub_grade', 'term', 'title',
       'verification_status', 'hardship_flag', 'disbursement_method',
       'debt_settlement_flag'],
      dtype='object')

In [13]:
# Split train and test data into numerical and categorical
num_X_train = X_train[num_cols]
num_X_test = X_test[num_cols]

cat_X_train = X_train[cat_cols]
cat_X_test = X_test[cat_cols]


print('Numerical data:')
print(num_X_train.shape)
print(num_X_test.shape)
print('\nCategorical data:')
print(cat_X_train.shape)
print(cat_X_test.shape)

Numerical data:
(979767, 46)
(326589, 46)

Categorical data:
(979767, 14)
(326589, 14)


### Treat missing numerical data

In [14]:
num_X_train.isnull().sum()

acc_now_delinq                    21
annual_inc                         3
avg_cur_bal                    52598
bc_open_to_buy                 47416
bc_util                        47960
delinq_2yrs                       21
delinq_amnt                       21
dti                              235
funded_amnt                        0
funded_amnt_inv                    0
inq_last_6mths                    22
installment                        0
int_rate                           0
loan_amnt                          0
mo_sin_old_il_acct             80090
mo_sin_old_rev_tl_op           52582
mo_sin_rcnt_rev_tl_op          52582
mort_acc                       37435
mths_since_recent_bc           46720
mths_since_recent_inq         129082
num_actv_bc_tl                 52581
num_actv_rev_tl                52581
num_bc_sats                    43881
num_bc_tl                      52581
num_il_tl                      52581
num_op_rev_tl                  52581
num_rev_accts                  52582
n

In [15]:
medians = pd.Series(num_X_train.median())

num_X_train = num_X_train.fillna(medians)
num_X_test = num_X_test.fillna(medians)

# num_X_train.isnull().sum()

### Pipeline numerical features

In [16]:
# Define pre-processors - remove outliers and scale
capper = outr.Winsorizer(distribution='skewed', tail='both', fold=1.5)
scaler = StandardScaler()

# Create pipeline
num_pipeline = Pipeline([('capper', capper, ('scaler', scaler))])

num_X_train = num_pipeline.fit_transform(num_X_train)
num_X_test = num_pipeline.transform(num_X_test)

# Retrieve features names
num_feature_cols = num_pipeline.named_steps['capper'].variables

# Cast to pandas array df
num_X_train = pd.DataFrame(num_X_train, columns=num_feature_cols)
num_X_test = pd.DataFrame(num_X_test, columns=num_feature_cols)

#num_X_train

### Treat missing categorical data

In [17]:
cat_X_train.fillna('other', inplace=True)
cat_X_test.fillna('other', inplace=True)

### Pipeline categorical features

In [18]:
# Define pre-processors

# Group frequency < 1% in category, “Rare”
encoder = RareLabelCategoricalEncoder(tol=0.01)

ohe = OneHotCategoricalEncoder() ####### top_categories=None,drop_last=True

# Create pipeline
cat_pipeline = Pipeline([('encoder', encoder, ('ohe', ohe))])

cat_X_train = cat_pipeline.fit_transform(cat_X_train)
cat_X_test = cat_pipeline.transform(cat_X_test)

# Reset indices to merge
cat_X_train.reset_index(drop=True, inplace=True)
cat_X_test.reset_index(drop=True, inplace=True)

In [32]:
cat_X_train.describe()

,addr_state_NY,addr_state_Rare,addr_state_CA,addr_state_FL,addr_state_TX,emp_length_other,emp_length_2 years,emp_length_5 years,emp_length_Rare,emp_length_10+ years,...,title_Credit card refinancing,verification_status_Source Verified,verification_status_Not Verified,verification_status_Verified,hardship_flag_N,hardship_flag_Y,disbursement_method_Cash,disbursement_method_DirectPay,debt_settlement_flag_N,debt_settlement_flag_Y
count,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,...,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000,979767.000000
mean,0.081971,0.619094,0.146354,0.070884,0.081697,0.057728,0.090322,0.062896,0.174545,0.328488,...,0.184550,0.385467,0.302215,0.312318,0.999999,0.000001,0.995515,0.004485,0.975547,0.024453
std,0.274320,0.485610,0.353461,0.256631,0.273903,0.233229,0.286643,0.242775,0.379577,0.469664,...,0.387932,0.486706,0.459218,0.463439,0.001010,0.001010,0.066818,0.066818,0.154450,0.154450
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
50%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
75%,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,...,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Encode target labels

In [19]:
le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_test = le.transform(y_test)

In [20]:
# Cast to pandas array
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

# label column
y_train.columns = ['labels']
y_test.columns = ['labels']

In [25]:
#y_train.head()
#len(y_train)

In [ ]:
pd.Series(le.inverse_transform(y_train.values.ravel())).head(10)

(979767,)

(979767, 1)

## Concatenate dataframes for export

In [37]:
final_train_df = pd.concat([num_X_train,cat_X_train,y_train],axis=1)
final_test_df = pd.concat([num_X_test,cat_X_test,y_test],axis=1)

print(final_train_df.shape)
print(final_test_df.shape)

(979767, 109)
(326589, 109)


In [38]:
final_train_df.head()

,acc_now_delinq,annual_inc,avg_cur_bal,bc_open_to_buy,bc_util,delinq_2yrs,delinq_amnt,dti,funded_amnt,funded_amnt_inv,...,verification_status_Source Verified,verification_status_Not Verified,verification_status_Verified,hardship_flag_N,hardship_flag_Y,disbursement_method_Cash,disbursement_method_DirectPay,debt_settlement_flag_N,debt_settlement_flag_Y,labels
0,-0.06916,-0.340928,-0.865093,0.911531,-1.567894,-0.429606,-0.038715,-0.520411,0.185192,0.188314,...,1,0,0,1,0,1,0,1,0,0
1,-0.06916,0.537220,-0.675828,-0.130907,-0.896770,1.090431,-0.038715,-1.311121,-0.505232,-0.501616,...,0,1,0,1,0,1,0,1,0,1
2,-0.06916,-0.456474,0.887956,-0.766508,1.421332,1.090431,-0.038715,-0.160679,0.018340,0.021581,...,1,0,0,1,0,1,0,1,0,1
3,-0.06916,-0.722229,0.981768,-0.754082,1.316129,-0.429606,-0.038715,-0.251780,-0.275091,-0.271640,...,0,1,0,1,0,1,0,1,0,0
4,-0.06916,-0.803111,-0.850328,-0.667596,0.412831,-0.429606,-0.038715,-0.679254,-1.011544,-1.007565,...,0,1,0,1,0,1,0,1,0,1


In [39]:
final_train_df.to_csv('Data/train.csv',index=False)
final_test_df.to_csv('Data/test.csv',index=False)